<a href="https://colab.research.google.com/github/Ashmaan-2006/facial-detection/blob/main/facial_keypoint_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

# Load dataset
df = pd.read_csv('/content/training.csv')  # Update path if needed

print("Dataset shape:", df.shape)
df.head()

# Drop incomplete rows
df = df.dropna()

# Process images
X = np.vstack(df['Image'].apply(lambda im: np.fromstring(im, sep=' ')).values)
X = X.reshape(-1, 96, 96, 1).astype('float32') / 255.0  # Normalize pixels

# Process keypoints
y = df[df.columns[:-1]].values
y = (y - 48) / 48  # Normalize to [-1, 1]

print(f"Images shape: {X.shape}")
print(f"Keypoints shape: {y.shape}")

# Attatined from sklearn.model_selection import train_test_split

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print(f"Training set shape: {X_train.shape}, Validation set shape: {X_val.shape}")

# Define CNN architecture
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(96,96,1)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(500, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(30)  # 15 keypoints x/y
])

# Compile model
model.compile(optimizer='adam', loss='mse')
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val))

# Plot loss curves
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

# Function to plot keypoints
def plot_keypoints(img, keypoints):
    plt.imshow(img.reshape(96,96), cmap='gray')
    keypoints = keypoints * 48 + 48  # Denormalize
    plt.scatter(keypoints[::2], keypoints[1::2], marker='x', color='red')

# Predict and visualize some examples
preds = model.predict(X_val)

plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3,3,i+1)
    plot_keypoints(X_val[i], preds[i])
plt.show()

# Save trained model
model.save('/content/facial_keypoint_model.h5')





FileNotFoundError: [Errno 2] No such file or directory: '/content/training.csv'